In [ ]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import spacy
import re
import os
import nltk
from nltk import sent_tokenize
import xlrd

# nlp = spacy.load('en')
# doc = nlp("I cured the illness. The illness was cured by me. A red cup. I ran quickly.")
# doc_list = list(doc)
# def flatten_subtree(st):
#    return ''.join([w.text_with_ws for w in list(st)]).strip()

### User Input

In [ ]:
# speaker_ids = speaker_ids
# xxx

speaker_ids = ["P1", "N1", "D1", "D2", "R1", "R2"]
speaker_ids = [x.lower() for x in speaker_ids]
speaker_ids

#### Create placeholders for results

In [ ]:
### Aggregate mentions of clinical concept (i.e. 1 number per consultation)
clin_concepts_number_per_consultation_speaker = {}

### Aggregate mentions of each clinical classification (separately) (i.e. >1 number per consultation)
clin_classes_type_number_per_consultation_speaker = {}

### Everything else in 1 large dictionary
per_speaker = {}
per_speaker['positive_sentiment_count']    = {}
per_speaker['negative_sentiment_count']    = {}
per_speaker['active_count']                = {}
per_speaker['passive_count']               = {}

per_speaker['pos_verb_count']                    = {}
per_speaker['pos_personal_pronoun_count']        = {}
per_speaker['pos_adverb_count']                  = {}
per_speaker['pos_adjective_count']               = {}


list_dict_parts = ["positive_sentiment_count", "negative_sentiment_count",
             "active_count",              "passive_count", 
             "pos_verb_count",      "pos_personal_pronoun_count",     
             "pos_adverb_count",    "pos_adjective_count" 
             ]


for x in list_dict_parts:
    type(per_speaker[x])
    print(per_speaker[x])

### Spacy preparation

In [ ]:
nlp = spacy.load('en')

#### Dictionary linking each clinical concept to its classification

In [ ]:
%run -i create_clin_concepts_dict.py

clin_concepts_dict

### Prepare the tokeniser

In [ ]:
%run -i sentiment_tokenize.py


### Create the 2 x lists of positive & negative words from General Inquirer

In [ ]:
%run -i sentiment_general_inquirer_import.ipynb

print("Positive sample")
positive[:5]
positive[-5:]

print()
print("Negative sample")
negative[:5]
negative[-5:]

### List of all classes (for dictionary of count of classes)

In [ ]:
list_classes = ['BIOSUB',
    'BODYLOC',
    'BODYMEAS',
    'DDF',
    'DIAGPROC',
    'DISORDER',
    'DOSEUNIT',
    'DRT',
    'DRUG',
    'FREQ',
    'LABTEST',
    'NUM',
    'PROBLEM',
    'THERPROCDEV',
    'TK',
    'TOD',
    'TUNIT',
    'UNIT',
    'RUT']


### START LOOP HERE - OVER EACH TRANSCRIPT SEPARATELY

In [ ]:
### PREAMBLE:
counter = 0
folder  = "sentiment_text_data"
number_of_files = len(os.listdir(folder))
print("Number of files in the directory: {}".format(number_of_files))


In [ ]:
type(list_dict_parts)

for x in list_dict_parts:
    print(x)
    type(x)

In [ ]:
### INITIATE LOOP PER FILE:
while counter < number_of_files:

    ### number of times any clinical concept mentioned: add to master dictionary for all transcripts
    clin_concepts_number_per_consultation_speaker[str(counter+1)] =  {}
    
    ### "per_speaker" dictionary for capturing everything EXCEPT the clin concepts per speaker.
    number_consult = str(counter+1)
    for x in list_dict_parts:
        per_speaker[x][number_consult] =  {}
        type(per_speaker[x][number_consult])

### INITIATE LOOP PER SPEAKER:
    for speaker_id in speaker_ids:

### print file name / number / speaker_id:
        file = os.listdir(folder)[counter]
        print()
        print("File number: {}".format(counter + 1))
        print("File name: {}".format(file))
        print("Speaker ID: {}".format(speaker_id))
        print()

### XXX XXX SORT OUT INDENTS BELOW.
### keep the lines relating to speaker of interest
        lines_to_keep  =  []
        with open(file, "r") as f:
            for line in f.readlines():
                print("This is a line of text")
                line = line.lower()
                print(line)
                print(type(line))
                if line.startswith(speaker_id):
                    lines_to_keep.append(line)

    
########################### Write all the lines in our list to a new text file called "speaker.txt"
        with open("speaker.txt", "w") as f:
            for line in lines_to_keep:
                f.write(line)


########################### Open the text file "speaker.txt" & tokenise into "tokens" (this removes punctuation such as ':' )
        speaker_opened = open("speaker.txt")  
        speaker_read =   speaker_opened.read()
        tokens  =   tokenize_only(speaker_read)
        speaker_sent_tokenize  = sent_tokenize(speaker_read)
    
        type(speaker_opened)
        type(speaker_read)
        type(tokens)
        type(speaker_sent_tokenize)


    ########################### Remove "n1" / "d1" / "p1" etc. from the list obviously:
        tokens  =  list(filter((speaker_id.lower).__ne__, tokens))

        print("Check: First 100 tokens")
        tokens
        tokens[:100]

        print("Check: Last 100 tokens")
        tokens[-100:]
        
        ######  Placeholder: Dictionary to count no. times each class occurs.
        ######  (will insert into "clin_classes_type_number_per_consultation" for every consultation)
        count_each_class_type_speaker = {}
        count_each_class_type_speaker[speaker_id] = {}
        for x in list_classes:
            count_each_class_type_speaker[speaker_id][x] = 0
        
            ### "per_speaker" dictionary for capturing everything EXCEPT the clin concepts per speaker.
        for x in list_dict_parts:
            per_speaker[x][number_consult][speaker_id] =  0

        # XXX - just so it runs quickly during development.
        my_text_strings = tokens[0:400]
        
        # my_text_strings  =  tokens

        
        ### run notebook code for: POS / positive and negative sentiment / active and passive run.
        % run -i loop_per_speaker_inner_loop.ipynb
        print("per_speaker['positive_sentiment_count']")
        print(per_speaker['positive_sentiment_count'])
        print("per_speaker['negative_sentiment_count']")
        print(per_speaker['negative_sentiment_count'])
        
        ### placeholders for results (variables relate to this combo of consultation / speaker)
        classifications_length = 0
        concepts_length        = 0
        all_classifications_consultation_prelim = []
    
        ### counter for the "for" loop over each token below
        token_number = 1
    
        ### loop over each token
        for mystring in my_text_strings:
            
            ### store the class & concept (if the token is a "clinical concept")
            classifications_consultation = [value for key, value in clin_concepts_dict.items() if mystring == key.lower()]
            concepts_consultation        = [key for key, value in clin_concepts_dict.items() if mystring == key.lower()]       
    
            ### update the number of clinical concepts per consultation / speaker
            classifications_length += len(classifications_consultation)
            concepts_length        += len(concepts_consultation)
            
            ### add the "classification" (e.g. treatment / problem) to the list of classifications for the specific transcript
            if classifications_consultation !=  []:
                all_classifications_consultation_prelim.extend(classifications_consultation)
            
            ### just a simple progress indicator for the token loop
            if token_number % 200 == 0:
                print("Progress tracking: File number {}, Token number {} (of a total of {} tokens)".format(counter+1, token_number, len(tokens)))
            
            ### and back to start of token loop
            token_number += 1
        
        
        print("")
        print("all_classifications_consultation_prelim: before split on hyphen")
        print(all_classifications_consultation_prelim[0:50])
    
        ### some tokens have multiple classes (e.g. "treatment-problem") - split on the hyphen to account for each class separately.
        all_classifications_consultation_prelim  = [i.split('-', 10)[:] for i in all_classifications_consultation_prelim]
    
        ### check that the split occurred ok and nothing is missing.
        print("")
        print("all_classifications_consultation_prelim: after split on hyphen")
        all_classifications_consultation_prelim[0:50]
    
        ### the final container for the classes for a consultation / speaker (i.e. put it in the right List form)
        all_classifications_consultation_final = []
        for x in all_classifications_consultation_prelim:
            all_classifications_consultation_final.extend(x)
        ### just show the first 20 (this confirms the content and the appropriate object type)
        all_classifications_consultation_final[0:20]
    
        ######### Freq Each Class Type: put into Dictionary (for this particular consultation / speaker)
        for x in all_classifications_consultation_final:
            count_each_class_type_speaker[speaker_id][x] +=1 
        print()
        print("count_each_class_type: i.e. the dictionary containing the number of each class for a consultation")
        count_each_class_type_speaker
            ## number of times each type of clinical classification mentioned: add to master dictionary for all transcripts
        clin_classes_type_number_per_consultation_speaker[str(counter+1)] = count_each_class_type_speaker            
        
        ######### Number of Concepts: put into Dictionary (for this particular consultation / speaker)
        print()
        print("classifications_length: for one transcript / speaker")
        classifications_length
        print()
        print("concepts_length: for one transcript")
        concepts_length
        ### number of times any clinical concept mentioned: add to master dictionary for all transcripts
        clin_concepts_number_per_consultation_speaker[str(counter+1)][speaker_id] = concepts_length


    counter  +=  1


### After loop -  look at results.

In [ ]:
per_speaker

In [ ]:

print("Number of clinical concepts per consultation")
clin_concepts_number_per_consultation_speaker

print("How many times each classification appeared in each transcript")
clin_classes_type_number_per_consultation_speaker


## Demo: method for adding new keys to a dictionary in a loop

In [ ]:
speak_ids = ["a", "b", "c"]
clin_classes_type_number_per_consultation_speaker = {}

for x in speak_ids:
    final = "List_" + x
    
    clin_classes_type_number_per_consultation_speaker[final] = {}
    clin_classes_type_number_per_consultation_speaker[final]["result_a"] = 1
    clin_classes_type_number_per_consultation_speaker[final]["result_b"] = 2

clin_classes_type_number_per_consultation_speaker

#### another little demo of how to automate the creation of dictionary components.

In [ ]:
speaker_ids = ["a", "b"]

list_classes = ['BIOSUB',
    'BODYLOC',
    'BODYMEAS',
    'UNIT']

count_each_class_type_speaker = {}

for speaker_id in speaker_ids:
        count_each_class_type_speaker[speaker_id] = {}
        for x in list_classes:
            count_each_class_type_speaker[speaker_id][x] = 0
count_each_class_type_speaker